In [21]:
import pandas as pd 


In [22]:
#/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/schedule_all_years.pkl
# read in the schedule data
schedule_all_years = pd.read_pickle(
    f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/schedule_all_years.pkl"
)
# read in defense 
defense_all_leagues = pd.read_pickle(
    f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/defense_data_merged.pkl"
)
# read in shooting data
shots_all_leagues = pd.read_pickle(
    f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/shooting_data_merged.pkl"
)   
# read in passing data
passing_all_leagues = pd.read_pickle(
    f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/passing_data_merged.pkl"
)   
# read in possession data
# /Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/passing_types_data_merged.pkl
type = pd.read_pickle(
    f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/passing_types_data_merged.pkl"
)

In [19]:
# merge schedule and defense on the match report. It should only have one match, no row extension, only new columns
# delete duplicate columns
defense_all_leagues = defense_all_leagues.drop_duplicates(subset=["match_report"])
merged = schedule_all_years.merge(
    defense_all_leagues,
    how="left",
    on="match_report",
)
merged = merged.drop_duplicates()

In [26]:
# isolate duplicates in match report as a new dataframe
defense_all_leagues_dups = defense_all_leagues[defense_all_leagues.duplicated(subset=["match_report"], keep=False)]
# print number

In [27]:
defense_all_leagues_dups 

,match_report,Int_home,Blocks_home,Clr_home,Err_home,season,Int_away,Blocks_away,Clr_away,Err_away
5774,/en/matches/98b24ea3/Granada-Alaves-September-...,6.0,8,20.0,0.0,2020-2021,8.0,9,17.0,0.0
5775,/en/matches/0a2446dc/Villarreal-Alaves-Septemb...,19.0,7,10.0,2.0,2020-2021,16.0,10,19.0,0.0
5776,/en/matches/606d1bce/Valladolid-Alaves-October...,11.0,6,15.0,0.0,2020-2021,12.0,12,18.0,0.0
5777,/en/matches/f0416d1e/Levante-Alaves-November-8...,18.0,4,4.0,0.0,2020-2021,19.0,11,34.0,0.0
5778,/en/matches/8514945b/Real-Madrid-Alaves-Novemb...,7.0,12,6.0,1.0,2020-2021,9.0,8,34.0,0.0
...,...,...,...,...,...,...,...,...,...,...
18891,/en/matches/a2ea8dfd/Auxerre-Toulouse-Septembe...,7.0,10,30.0,0.0,2025-2026,5.0,10,28.0,0.0
18892,/en/matches/60ecc64f/Lyon-Toulouse-October-5-2...,7.0,13,14.0,1.0,2025-2026,15.0,6,34.0,1.0
18893,/en/matches/fe7036bb/Monaco-Toulouse-October-2...,12.0,12,41.0,1.0,2025-2026,9.0,16,11.0,0.0
18894,/en/matches/3b9a668d/Lorient-Toulouse-November...,11.0,9,16.0,1.0,2025-2026,12.0,7,25.0,1.0
